In [9]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import re

# Helper Functions

In [50]:

def to_dates_list(dates_dictionary):
    dates_list = []
    for date, date_range in dates_dictionary.items():
        if(date_range==0):
            continue
        dates_list.append(date)
        i = 2
        while(date_range-i>=0):
            new_date = pd.to_datetime(date) + pd.DateOffset(days=i-1)
            dates_list.append(new_date.strftime("%Y-%m-%d"))
            i = i+1
    #to avoid overlapping dates        
    return list(set(dates_list))


# Data Parsing Function 

This is a pilot for creating a function, which recieves as an input a date and intially returns the climate values
It also has an example of finding the average cases based on range of dates.


In [91]:
file_path = 'sample_data.xlsx'
file_path2 = 'CSSE_confirmed_with_GSOD_wx.csv'
date_range_dict = {'2020-03-27':1,'2020-02-14':1}



def DataParser(file_path, date_range_dict, incubation_duration=5, region = None, attr_extract=None,_attr_date = 'date_ymd', _attr_country = 'Country/Region'):
    
               
    if attr_extract == None:
        return None
               
    #FileOpen - according to file extension
    if file_path.endswith(".xlsx"):
        xlsx = pd.ExcelFile(file_path)
        df = pd.read_excel(xlsx, sheet_name='Sheet1')
    elif file_path.endswith(".csv"):
        df = pd.read_csv(file_path, parse_dates=['date_ymd'])
    
    
     #RegionSelection - if region parameter is not None, choose the region
    df_extract = df
    if region != None:
        mask_prev = (df_extract[_attr_country] == None)
        for reg in region:
            mask = mask_prev | (df_extract[_attr_country] == reg) 
            mask_prev = mask    
        df_extract = df.loc[mask]
    else:
        df_extract = df
    
    #DateParsing - create a mask to choose the desired dates (According to list and incubation period)
    df_final = pd.DataFrame() #creates a new dataframe that's empty
    date_list = to_dates_list(date_range_dict)
    for date in date_list:
        #get the incubation period of the date
        curr_date_df = df_extract.loc[:,_attr_date: _attr_country]
        end_date = (pd.to_datetime(date)-timedelta(days=incubation_duration)) 
        mask = (df_extract[_attr_date] >= end_date) & (df_extract[_attr_date] <= date)
        incubation_df_extract = df_extract.loc[mask] #extracts the incubation period of the date
        
        att_df_final = incubation_df_extract.groupby([_attr_country],as_index=False).agg(attr_extract)
        att_df_final.insert(0,_attr_date,date,True)
        df_final = df_final.append(att_df_final, ignore_index = True) # ignoring index is optional
    
    return df_final

df = DataParser(file_path2,date_range_dict,region=['United Arab Emirates','Antigua and Barbuda'], attr_extract={'confirmed_cases':['mean','min'],'mean_temp': ['max']})
df

date_ymd        Country/Region confirmed_cases      mean_temp
                                               mean  min       max
0  2020-02-14   Antigua and Barbuda        0.000000    0       NaN
1  2020-02-14  United Arab Emirates        7.833333    7    73.975
2  2020-03-27   Antigua and Barbuda        4.000000    1       NaN
3  2020-03-27  United Arab Emirates      278.333333  153    76.500

In [86]:
end_date = (pd.to_datetime(date)+timedelta(days=5)) 
mask = (df['Date'] >= date) & (df['Date'] <= end_date)
df_extract = df.loc[mask]
df_extract

ValueError: If using all scalar values, you must pass an index

In [42]:
df1 = df_extract['Confirmed Cases'].tolist()
df1

NameError: name 'df_extract' is not defined